### Required Libraries

### Load Dataset

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

In [4]:
def map_weight(value):
    if (value >= '2022-01') and (value <= '2022-06'):
        return 2
    elif (value >= '2021-07') and (value <= '2021-12'):
        return 1
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


### Drop Null target records

In [9]:
model_data['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
NaN    424397
0.0     44727
1.0      9204
Name: count, dtype: int64

In [10]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
data.shape

(53931, 2832)

### Split the Time interval

In [12]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [13]:
model_df1.shape, oot_df1.shape

((39010, 2832), (12128, 2832))

In [14]:
features_list =[
    
#Bureau Intutive features
'Num_Enq_all_loans_last_9M_3M',
'Utilization_all',
'total_emi',
'Utilization_CREDIT CARD_opened_last_36M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'num_CREDIT CARD_enq_last_6M',
'num_CONSUMER LOAN_enq_last_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'highest_sanction_amount_credit_cards',
'num_CONSUMER LOAN_enq_last_6M',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M',
'Age_of_oldest_open_account',
    
#Banking Intutive features

'num_credit_gt_salary',    
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',    
'ratio_max_credit_to_total_credit_M2',
'spend_vs_salary_ratio',
'bounced_transactions_count',
'ratio_total_debit_to_max_credit_M2_M3',
'min_balance_last_3_months',
'min_balance_M2',
'days_balance_lt_5000_M3',
'days_balance_lt_10000_M3',
'Max_Credit_Avg_M1_M2_M3',
'Max_Credit_Diff_M1_M2_M3',
'amount_debits_last_2_months',
'sum_insurance_debit',
'avg_num_debits_last_2_months',
'upi_trans_debit_sum',
'cash_withdrawal_sum',
    
#new banking features
'diff_btw_stated_salary_vs_actual_salary',
'balance_6th_day_M2',
'ratio_6th_day_balance_to_max_credit_M3',

#Loan application Features

'deviation_from_median_income/standard_deviation_income#last_365_Days',
# 'yearOfBirth_mean',
'loan_amount_to_income#last_365_Days',



#PG synthetic Features
'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 
    
#ID Features
'dm206s',
'g406s',
'balmag01',

# # Reconcilated Features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
# 'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'Num_of_installment_loan_opened_last_6M',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Weight_vintage',
'month_year',
'90_in_24mob'
              
]




In [15]:
len(features_list)

53

In [16]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [17]:
oot_df = oot_df1[features_list]

In [18]:
model_df.shape, oot_df.shape

((39010, 53), (12128, 53))

In [19]:
model_df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    32389
1.0     6621
Name: count, dtype: int64

In [20]:
oot_df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    10039
1.0     2089
Name: count, dtype: int64

In [21]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

In [22]:
X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

In [23]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3023657/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [24]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3023657/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [25]:
X_oot.shape,y_oot.shape

((12128, 51), (12128,))

### Split Model dataset into Train and Test

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [27]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 51) (4682, 51) (15604, 51)


In [28]:
X_train.shape, X_test.shape

((23406, 51), (15604, 51))

In [29]:
#Get vintage weight values
train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values


In [30]:
oot_weights = X_oot['Weight_vintage'].values

In [31]:
X_train_new = X_train_new.drop(columns=['Weight_vintage'])

In [32]:
X_eval = X_eval.drop(columns=['Weight_vintage'])

In [33]:

X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])


In [34]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [35]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
    'max_depth': 2,
    'subsample': 0.3,
    'colsample_bytree': 0.3,
    'lambda': 12.6,
    'alpha': 4.8,
    'gamma': 5.6,
    'scale_pos_weight': 3.8,
    'random_state': 24   
}


evals = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 500,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.61494	eval-auc:0.59547
[10]	train-auc:0.66692	eval-auc:0.65214
[20]	train-auc:0.67684	eval-auc:0.66001
[30]	train-auc:0.68064	eval-auc:0.66298
[40]	train-auc:0.68163	eval-auc:0.66326
[50]	train-auc:0.68362	eval-auc:0.66392
[60]	train-auc:0.68444	eval-auc:0.66391
[70]	train-auc:0.68483	eval-auc:0.66445
[80]	train-auc:0.68554	eval-auc:0.66477
[90]	train-auc:0.68513	eval-auc:0.66406
[100]	train-auc:0.68711	eval-auc:0.66546
[110]	train-auc:0.68824	eval-auc:0.66582
[120]	train-auc:0.68924	eval-auc:0.66638
[130]	train-auc:0.69027	eval-auc:0.66716
[140]	train-auc:0.69155	eval-auc:0.66732
[150]	train-auc:0.69257	eval-auc:0.66823
[160]	train-auc:0.69320	eval-auc:0.66847
[170]	train-auc:0.69372	eval-auc:0.66852
[180]	train-auc:0.69473	eval-auc:0.66914
[190]	train-auc:0.69564	eval-auc:0.66945
[200]	train-auc:0.69596	eval-auc:0.66951
[210]	train-auc:0.69689	eval-auc:0.66988
[220]	train-auc:0.69703	eval-auc:0.66981
[230]	train-auc:0.69752	eval-auc:0.66994
[240]	train-auc:0.69822	eva

In [36]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.70, Gini: 0.40, KS: 0.29
Eval AUC: 0.68, Gini: 0.37, KS: 0.26
Test AUC: 0.68, Gini: 0.35, KS: 0.26
OOT AUC: 0.69, Gini: 0.38, KS: 0.29


In [37]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.399835         0.642667        0.821125        0.580037   1213   
9          0.262984         0.543803        0.579970        0.511043   1213   
8          0.229184         0.484137        0.511014        0.459938   1213   
7          0.199670         0.440786        0.459933        0.422765   1212   
6          0.148392         0.406513        0.422744        0.391402   1213   
5          0.139324         0.374850        0.391394        0.359443   1213   
4          0.108911         0.342553        0.359435        0.324189   1212   
3          0.099753         0.305689        0.324189        0.287523   1213   
2          0.089860         0.269371        0.287433        0.251024   1213   
1          0.044518         0.226167        0.251017        0.167152   1213   

        label_good  label_bad  
Decile                         
10             728        485  
9              894        319  
8              935        278  
7              970        242  
6             1033        180  
5             1044        169  
4             1080        132  
3             1092        121  
2             1104        109  
1             1159         54

In [39]:
#Save the model
xgb_model.save_model('LeoPayu_BBILAP_weightVintage_90_in_24mob.json')
print('Model Saved')

Model Saved
